# Reducing category size
Fewer categorical values -> Less split possibilities in tree based learnes -> Less overfit and quicker learning?

## Load data

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [3]:
cat_cols = [x for x in train.columns if x.startswith('cat')]
cat_cols

['cat0',
 'cat1',
 'cat2',
 'cat3',
 'cat4',
 'cat5',
 'cat6',
 'cat7',
 'cat8',
 'cat9',
 'cat10',
 'cat11',
 'cat12',
 'cat13',
 'cat14',
 'cat15',
 'cat16',
 'cat17',
 'cat18']

In [4]:
x_train = train.drop(columns=['id','target'])
y_train = train['target']

## Reduction of columns in one hot encoding by category size reduction

In [5]:
from categorical_transform import CategoricalTransform, OneHotTransform
from sklearn.pipeline import Pipeline
import pandas as pd

In [18]:
min_cat_size = [0, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]
data_size = []
for min_data_portion in min_cat_size:
    ct = CategoricalTransform(cat_cols, min_data_portion = min_data_portion)
    pipe = Pipeline([('categorical_transform', ct), 
                    ('one hot', OneHotTransform())])
    data_size.append(pipe.fit_transform(x_train).shape[1])
pd.DataFrame(dict(min_cat_size=min_cat_size, num_columns=data_size))

,min_cat_size,num_columns
0,0.0000,634
1,0.0005,399
2,0.0010,297
3,0.0050,186
4,0.0100,150
5,0.0500,70
6,0.1000,48


## Analysis Tools

In [26]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
kfold = KFold(n_splits=5, shuffle=True)

In [29]:
def analyse_cat_size(pipe):
    gscv = GridSearchCV(pipe, dict(trans__min_data_portion=min_cat_size), cv=kfold, scoring='roc_auc', verbose=1)
    gscv.fit(x_train, y_train)
    return pd.DataFrame(dict(cat_size = min_cat_size, 
                  fit_time=gscv.cv_results_['mean_fit_time'], 
                  auc=gscv.cv_results_['mean_test_score']))

## Lightgbm default model performance with different min_cat_sizes

In [30]:
from lightgbm.sklearn import LGBMClassifier

In [31]:
lightgbm_pipe = Pipeline([("trans", CategoricalTransform(cat_cols)),
                          ("lgbm", LGBMClassifier(n_jobs = -2))])

In [32]:
analyse_cat_size(lightgbm_pipe)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  1.3min finished


,cat_size,fit_time,auc
0,0.0000,2.075828,0.891309
1,0.0005,2.044400,0.891471
2,0.0010,2.022325,0.891758
3,0.0050,2.021178,0.891557
4,0.0100,2.031074,0.891152
5,0.0500,1.936202,0.887975
6,0.1000,1.898409,0.886370


## Catboost performance

In [34]:
from catboost import CatBoostClassifier
from categorical_transform import IntegerCategoricalTransform
cbc = CatBoostClassifier(cat_features=cat_cols, thread_count=6, verbose=0)
catboost_pipe = Pipeline([("trans", IntegerCategoricalTransform(cat_cols)),
                          ("catboost", cbc)])

In [35]:
analyse_cat_size(catboost_pipe)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Learning rate set to 0.106975
0:	learn: 0.6227549	total: 302ms	remaining: 5m 1s
1:	learn: 0.5664109	total: 522ms	remaining: 4m 20s
2:	learn: 0.5259600	total: 721ms	remaining: 3m 59s
3:	learn: 0.4939128	total: 904ms	remaining: 3m 45s
4:	learn: 0.4704020	total: 1.08s	remaining: 3m 34s
5:	learn: 0.4506750	total: 1.26s	remaining: 3m 29s
6:	learn: 0.4362881	total: 1.47s	remaining: 3m 28s
7:	learn: 0.4244446	total: 1.68s	remaining: 3m 28s
8:	learn: 0.4152073	total: 1.8s	remaining: 3m 18s
9:	learn: 0.4081617	total: 2s	remaining: 3m 18s
10:	learn: 0.4020839	total: 2.2s	remaining: 3m 17s
11:	learn: 0.3972343	total: 2.34s	remaining: 3m 12s
12:	learn: 0.3933719	total: 2.53s	remaining: 3m 12s
13:	learn: 0.3902164	total: 2.73s	remaining: 3m 12s
14:	learn: 0.3864064	total: 2.92s	remaining: 3m 11s
15:	learn: 0.3835385	total: 3.16s	remaining: 3m 14s
16:	learn: 0.3816269	total: 3.4s	remaining: 3m 16s
17:	learn: 0.3795311	total: 3.64s	remaining: 3m 18s
18:	learn: 0.3777146	total: 3.89s	remaining: 3m 20s

158:	learn: 0.3470783	total: 34.6s	remaining: 3m 3s
159:	learn: 0.3469861	total: 34.8s	remaining: 3m 2s
160:	learn: 0.3469157	total: 35.1s	remaining: 3m 2s
161:	learn: 0.3468739	total: 35.3s	remaining: 3m 2s
162:	learn: 0.3467855	total: 35.5s	remaining: 3m 2s
163:	learn: 0.3467182	total: 35.7s	remaining: 3m 1s
164:	learn: 0.3466319	total: 35.9s	remaining: 3m 1s
165:	learn: 0.3465261	total: 36.1s	remaining: 3m 1s
166:	learn: 0.3464735	total: 36.3s	remaining: 3m
167:	learn: 0.3464007	total: 36.5s	remaining: 3m
168:	learn: 0.3463332	total: 36.7s	remaining: 3m
169:	learn: 0.3462429	total: 36.9s	remaining: 3m
170:	learn: 0.3461801	total: 37.1s	remaining: 3m
171:	learn: 0.3461145	total: 37.4s	remaining: 2m 59s
172:	learn: 0.3460709	total: 37.6s	remaining: 2m 59s
173:	learn: 0.3459725	total: 37.8s	remaining: 2m 59s
174:	learn: 0.3458872	total: 38s	remaining: 2m 59s
175:	learn: 0.3458334	total: 38.2s	remaining: 2m 58s
176:	learn: 0.3457737	total: 38.4s	remaining: 2m 58s
177:	learn: 0.3457015	t

314:	learn: 0.3387379	total: 1m 8s	remaining: 2m 29s
315:	learn: 0.3386913	total: 1m 9s	remaining: 2m 29s
316:	learn: 0.3386330	total: 1m 9s	remaining: 2m 29s
317:	learn: 0.3386003	total: 1m 9s	remaining: 2m 29s
318:	learn: 0.3385519	total: 1m 9s	remaining: 2m 28s
319:	learn: 0.3385093	total: 1m 9s	remaining: 2m 28s
320:	learn: 0.3384697	total: 1m 10s	remaining: 2m 28s
321:	learn: 0.3384282	total: 1m 10s	remaining: 2m 28s
322:	learn: 0.3383949	total: 1m 10s	remaining: 2m 28s
323:	learn: 0.3383509	total: 1m 10s	remaining: 2m 27s
324:	learn: 0.3383319	total: 1m 11s	remaining: 2m 27s
325:	learn: 0.3382809	total: 1m 11s	remaining: 2m 27s
326:	learn: 0.3382309	total: 1m 11s	remaining: 2m 27s
327:	learn: 0.3381656	total: 1m 11s	remaining: 2m 26s
328:	learn: 0.3381068	total: 1m 11s	remaining: 2m 26s
329:	learn: 0.3380660	total: 1m 12s	remaining: 2m 26s
330:	learn: 0.3380011	total: 1m 12s	remaining: 2m 26s
331:	learn: 0.3379603	total: 1m 12s	remaining: 2m 26s
332:	learn: 0.3379313	total: 1m 12

...

466:	learn: 0.3334386	total: 2m 14s	remaining: 2m 33s
467:	learn: 0.3333995	total: 2m 14s	remaining: 2m 32s
468:	learn: 0.3333756	total: 2m 14s	remaining: 2m 32s
469:	learn: 0.3333504	total: 2m 15s	remaining: 2m 32s
470:	learn: 0.3333279	total: 2m 15s	remaining: 2m 32s
471:	learn: 0.3333040	total: 2m 15s	remaining: 2m 31s
472:	learn: 0.3332709	total: 2m 15s	remaining: 2m 31s
473:	learn: 0.3332394	total: 2m 16s	remaining: 2m 31s
474:	learn: 0.3332190	total: 2m 16s	remaining: 2m 30s
475:	learn: 0.3332044	total: 2m 16s	remaining: 2m 30s
476:	learn: 0.3331749	total: 2m 17s	remaining: 2m 30s
477:	learn: 0.3331543	total: 2m 17s	remaining: 2m 30s
478:	learn: 0.3331179	total: 2m 17s	remaining: 2m 29s
479:	learn: 0.3330908	total: 2m 17s	remaining: 2m 29s
480:	learn: 0.3330657	total: 2m 18s	remaining: 2m 29s
481:	learn: 0.3330401	total: 2m 18s	remaining: 2m 28s
482:	learn: 0.3329994	total: 2m 18s	remaining: 2m 28s
483:	learn: 0.3329717	total: 2m 19s	remaining: 2m 28s
484:	learn: 0.3329543	total:

619:	learn: 0.3292645	total: 2m 58s	remaining: 1m 49s
620:	learn: 0.3292324	total: 2m 58s	remaining: 1m 49s
621:	learn: 0.3292110	total: 2m 58s	remaining: 1m 48s
622:	learn: 0.3291816	total: 2m 59s	remaining: 1m 48s
623:	learn: 0.3291460	total: 2m 59s	remaining: 1m 48s
624:	learn: 0.3291253	total: 2m 59s	remaining: 1m 47s
625:	learn: 0.3291133	total: 3m	remaining: 1m 47s
626:	learn: 0.3291046	total: 3m	remaining: 1m 47s
627:	learn: 0.3290966	total: 3m	remaining: 1m 47s
628:	learn: 0.3290463	total: 3m 1s	remaining: 1m 46s
629:	learn: 0.3290121	total: 3m 1s	remaining: 1m 46s
630:	learn: 0.3289780	total: 3m 1s	remaining: 1m 46s
631:	learn: 0.3289421	total: 3m 1s	remaining: 1m 45s
632:	learn: 0.3289134	total: 3m 2s	remaining: 1m 45s
633:	learn: 0.3288842	total: 3m 2s	remaining: 1m 45s
634:	learn: 0.3288687	total: 3m 2s	remaining: 1m 45s
635:	learn: 0.3288551	total: 3m 2s	remaining: 1m 44s
636:	learn: 0.3288180	total: 3m 3s	remaining: 1m 44s
637:	learn: 0.3287717	total: 3m 3s	remaining: 1m 

772:	learn: 0.3255329	total: 3m 42s	remaining: 1m 5s
773:	learn: 0.3255193	total: 3m 43s	remaining: 1m 5s
774:	learn: 0.3254784	total: 3m 43s	remaining: 1m 4s
775:	learn: 0.3254616	total: 3m 43s	remaining: 1m 4s
776:	learn: 0.3254436	total: 3m 43s	remaining: 1m 4s
777:	learn: 0.3254106	total: 3m 44s	remaining: 1m 3s
778:	learn: 0.3253873	total: 3m 44s	remaining: 1m 3s
779:	learn: 0.3253469	total: 3m 44s	remaining: 1m 3s
780:	learn: 0.3253368	total: 3m 45s	remaining: 1m 3s
781:	learn: 0.3253022	total: 3m 45s	remaining: 1m 2s
782:	learn: 0.3252735	total: 3m 45s	remaining: 1m 2s
783:	learn: 0.3252558	total: 3m 45s	remaining: 1m 2s
784:	learn: 0.3252238	total: 3m 46s	remaining: 1m 1s
785:	learn: 0.3251898	total: 3m 46s	remaining: 1m 1s
786:	learn: 0.3251711	total: 3m 46s	remaining: 1m 1s
787:	learn: 0.3251384	total: 3m 47s	remaining: 1m 1s
788:	learn: 0.3251233	total: 3m 47s	remaining: 1m
789:	learn: 0.3251210	total: 3m 47s	remaining: 1m
790:	learn: 0.3250995	total: 3m 47s	remaining: 1m
79

928:	learn: 0.3221076	total: 4m 28s	remaining: 20.5s
929:	learn: 0.3220854	total: 4m 28s	remaining: 20.2s
930:	learn: 0.3220593	total: 4m 29s	remaining: 19.9s
931:	learn: 0.3220345	total: 4m 29s	remaining: 19.7s
932:	learn: 0.3219987	total: 4m 29s	remaining: 19.4s
933:	learn: 0.3219549	total: 4m 29s	remaining: 19.1s
934:	learn: 0.3219481	total: 4m 30s	remaining: 18.8s
935:	learn: 0.3219162	total: 4m 30s	remaining: 18.5s
936:	learn: 0.3218996	total: 4m 30s	remaining: 18.2s
937:	learn: 0.3218753	total: 4m 31s	remaining: 17.9s
938:	learn: 0.3218541	total: 4m 31s	remaining: 17.6s
939:	learn: 0.3218311	total: 4m 31s	remaining: 17.3s
940:	learn: 0.3218154	total: 4m 32s	remaining: 17.1s
941:	learn: 0.3217943	total: 4m 32s	remaining: 16.8s
942:	learn: 0.3217678	total: 4m 32s	remaining: 16.5s
943:	learn: 0.3217616	total: 4m 32s	remaining: 16.2s
944:	learn: 0.3217476	total: 4m 33s	remaining: 15.9s
945:	learn: 0.3217436	total: 4m 33s	remaining: 15.6s
946:	learn: 0.3217314	total: 4m 33s	remaining:

,cat_size,fit_time,auc
0,0.0000,232.666317,0.894349
1,0.0005,230.426964,0.894257
2,0.0010,229.182759,0.894283
3,0.0050,226.390067,0.893971
4,0.0100,224.918956,0.893505
5,0.0500,221.400715,0.891075
6,0.1000,188.128840,0.889794


## Default xgboost, integer encoding

In [39]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_jobs=-2, eval_metric='auc', use_label_encoder=False)
from categorical_transform import IntegerCategoricalTransform
from sklearn.pipeline import Pipeline
xgb_pipe = Pipeline([('trans',IntegerCategoricalTransform(cat_cols=cat_cols)),('xgboost', xgb)])

In [40]:
analyse_cat_size(xgb_pipe)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed: 12.5min finished


,cat_size,fit_time,auc
0,0.0000,21.494866,0.888903
1,0.0005,21.419623,0.888586
2,0.0010,21.400133,0.888835
3,0.0050,21.348674,0.889015
4,0.0100,21.428119,0.888676
5,0.0500,20.982548,0.886909
6,0.1000,20.753938,0.886223


## Default xgboost, one-hot encoding

In [41]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_jobs=-2, eval_metric='auc', use_label_encoder=False)
from categorical_transform import CategoricalTransform, OneHotTransform
from sklearn.pipeline import Pipeline
xgb_pipe = Pipeline([('trans',CategoricalTransform(cat_cols=cat_cols)),
                     ('oht',OneHotTransform()),
                     ('xgboost', xgb)])

In [42]:
analyse_cat_size(xgb_pipe)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed: 27.5min finished


,cat_size,fit_time,auc
0,0.0000,97.242565,0.889612
1,0.0005,65.530826,0.889513
2,0.0010,52.172046,0.889677
3,0.0050,37.558545,0.889735
4,0.0100,33.044849,0.889432
5,0.0500,22.379101,0.887205
6,0.1000,19.301642,0.885732


One hot slightly better than ordinal encoding. Reducing category size speeds up training significantly.

## Randomforest default integer encoding

In [44]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-2)
from categorical_transform import IntegerCategoricalTransform
from sklearn.pipeline import Pipeline
rf_pipe = Pipeline([('trans',IntegerCategoricalTransform(cat_cols=cat_cols)),('rf', rf)])

In [45]:
analyse_cat_size(rf_pipe)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed: 13.0min finished


,cat_size,fit_time,auc
0,0.0000,22.656157,0.883611
1,0.0005,22.240469,0.883712
2,0.0010,22.187217,0.883679
3,0.0050,21.972300,0.883936
4,0.0100,21.822261,0.883759
5,0.0500,21.092638,0.882684
6,0.1000,20.234398,0.882089


## Randomforest one hot encoding

In [47]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-2)
from categorical_transform import CategoricalTransform, OneHotTransform
from sklearn.pipeline import Pipeline
rf_pipe = Pipeline([('trans',CategoricalTransform(cat_cols=cat_cols)),
                    ('oht', OneHotTransform()),
                    ('rf', rf)])

In [48]:
analyse_cat_size(rf_pipe)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed: 21.7min finished


,cat_size,fit_time,auc
0,0.0000,58.610078,0.887117
1,0.0005,47.141677,0.887066
2,0.0010,41.913860,0.886667
3,0.0050,33.468690,0.886699
4,0.0100,30.774355,0.886101
5,0.0500,22.898024,0.883393
6,0.1000,19.931783,0.882050


One hot slightly better than ordinal encoding. Reducing category size speeds up training slightly.